In [1]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import StratifiedKFold

from multi_imbalance.utils import data
from multi_imbalance.ensemble.ecoc import ECOC


In [2]:
minority = dict()
minority['1czysty-cut'] = [1, 2]
minority['2delikatne-cut'] = [1, 2]
minority['3mocniej-cut'] = [1, 2]
minority['4delikatne-bezover-cut'] = [1, 2]
minority['balance-scale'] = [0]
minority['car'] = [3, 1]
minority['cleveland'] = [4, 3, 2, 1]
minority['cleveland_v2'] = [3, 2, 1]
minority['cmc'] = [1]
minority['dermatology'] = [5]
minority['flare'] = [4, 1]
minority['glass'] = [4, 2, 5]
minority['hayes-roth'] = [0]
minority['new_ecoli'] = [3, 2, 4]
minority['new_led7digit'] = [1, 4]
minority['new_vehicle'] = [2, 0]
minority['new_winequality-red'] = [3, 2]
minority['new_yeast'] = [2, 3, 4, 5, 6]
minority['thyroid-newthyroid'] = [2, 1]

In [3]:
datasets = data.load_arff_datasets()
datasets_names = [dsn for dsn, _ in datasets.items()]

binary_classifiers = ['tree','KNN', 'NB']
binary_oversamplings = [None, 'globalCS', 'SMOTE', 'SOUP']
encodings = ['sparse', 'dense', 'complete', 'OVO', 'OVA']
weights = [None, 'acc', 'avg_tpr_min']

results_g_mean = dict()
results_gmean_min = dict()

methods=[(bc,encoding, bo, w) for bc in binary_classifiers for encoding in encodings for bo in binary_oversamplings for w in weights]

for res in (results_g_mean, results_gmean_min):
    res['dataset'] = datasets_names
    for method in methods:
        res[method] = list()

In [5]:
for dataset_name, dataset_values in datasets.items():
    X, y = dataset_values.data, dataset_values.target
    
    for col_idx, col in enumerate(X.T):
        if len(set(col)) > 2:
            scaled = (col - np.mean(col)) / np.std(col)
            X[:, col_idx] = scaled
    
    dataset_name = dataset_name[5:]
    
    for binary_classifier, encoding, oversample_binary, weights in methods:
                method = (binary_classifier, encoding, oversample_binary, weights)
                print(dataset_name, method)
                g_mean, g_mean_min = list(), list()

                for i in range(10):
                    skf = StratifiedKFold(n_splits=5, random_state=i, shuffle=True)

                    for train_index, test_index in skf.split(X, y):
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]

                        ecoc = ECOC(binary_classifier=binary_classifier, preprocessing=oversample_binary, n_neighbors=3,
                                  encoding=encoding, weights=weights)
                        ecoc.fit(X_train, y_train, minority[dataset_name])
                        y_pred = ecoc.predict(X_test)

                        g_mean.append(geometric_mean_score(y_test, y_pred, correction=0.001))
                        g_mean_min.append(geometric_mean_score(y_test, y_pred, labels=minority[dataset_name], correction=0.001))
                                        
                results_g_mean[method].append(round(np.mean(g_mean), 3))
                results_gmean_min[method].append(round(np.mean(g_mean_min), 3))
                
                print('g-mean:', results_g_mean[method][-1])
                print('gmean-min:', results_gmean_min[method][-1]) 

1czysty-cut ('tree', 'sparse', None, None)
g-mean: 0.94
gmean-min: 0.929
1czysty-cut ('tree', 'sparse', None, 'acc')
g-mean: 0.932
gmean-min: 0.92
1czysty-cut ('tree', 'sparse', None, 'avg_tpr_min')
g-mean: 0.759
gmean-min: 0.97
1czysty-cut ('tree', 'sparse', 'globalCS', None)
g-mean: 0.945
gmean-min: 0.935
1czysty-cut ('tree', 'sparse', 'globalCS', 'acc')
g-mean: 0.942
gmean-min: 0.935
1czysty-cut ('tree', 'sparse', 'globalCS', 'avg_tpr_min')
g-mean: 0.76
gmean-min: 0.972
1czysty-cut ('tree', 'sparse', 'SMOTE', None)
g-mean: 0.953
gmean-min: 0.949
1czysty-cut ('tree', 'sparse', 'SMOTE', 'acc')
g-mean: 0.957
gmean-min: 0.956
1czysty-cut ('tree', 'sparse', 'SMOTE', 'avg_tpr_min')
g-mean: 0.764
gmean-min: 0.977
1czysty-cut ('tree', 'sparse', 'SOUP', None)
g-mean: 0.956
gmean-min: 0.975
1czysty-cut ('tree', 'sparse', 'SOUP', 'acc')
g-mean: 0.952
gmean-min: 0.975
1czysty-cut ('tree', 'sparse', 'SOUP', 'avg_tpr_min')
g-mean: 0.754
gmean-min: 0.992
1czysty-cut ('tree', 'dense', None, None)
g

g-mean: 0.974
gmean-min: 0.985
1czysty-cut ('KNN', 'OVO', 'SOUP', None)
g-mean: 0.955
gmean-min: 1.0
1czysty-cut ('KNN', 'OVO', 'SOUP', 'acc')
g-mean: 0.943
gmean-min: 1.0
1czysty-cut ('KNN', 'OVO', 'SOUP', 'avg_tpr_min')
g-mean: 0.942
gmean-min: 1.0
1czysty-cut ('KNN', 'OVA', None, None)
g-mean: 0.959
gmean-min: 0.952
1czysty-cut ('KNN', 'OVA', None, 'acc')
g-mean: 0.968
gmean-min: 0.964
1czysty-cut ('KNN', 'OVA', None, 'avg_tpr_min')
g-mean: 0.968
gmean-min: 0.964
1czysty-cut ('KNN', 'OVA', 'globalCS', None)
g-mean: 0.965
gmean-min: 0.975
1czysty-cut ('KNN', 'OVA', 'globalCS', 'acc')
g-mean: 0.974
gmean-min: 0.987
1czysty-cut ('KNN', 'OVA', 'globalCS', 'avg_tpr_min')
g-mean: 0.974
gmean-min: 0.987
1czysty-cut ('KNN', 'OVA', 'SMOTE', None)
g-mean: 0.967
gmean-min: 0.966
1czysty-cut ('KNN', 'OVA', 'SMOTE', 'acc')
g-mean: 0.972
gmean-min: 0.983
1czysty-cut ('KNN', 'OVA', 'SMOTE', 'avg_tpr_min')
g-mean: 0.975
gmean-min: 0.985
1czysty-cut ('KNN', 'OVA', 'SOUP', None)
g-mean: 0.961
gmean-m

g-mean: 0.7
gmean-min: 0.622
2delikatne-cut ('tree', 'complete', 'SMOTE', None)
g-mean: 0.687
gmean-min: 0.596
2delikatne-cut ('tree', 'complete', 'SMOTE', 'acc')
g-mean: 0.73
gmean-min: 0.668
2delikatne-cut ('tree', 'complete', 'SMOTE', 'avg_tpr_min')
g-mean: 0.707
gmean-min: 0.634
2delikatne-cut ('tree', 'complete', 'SOUP', None)
g-mean: 0.784
gmean-min: 0.764
2delikatne-cut ('tree', 'complete', 'SOUP', 'acc')
g-mean: 0.78
gmean-min: 0.771
2delikatne-cut ('tree', 'complete', 'SOUP', 'avg_tpr_min')
g-mean: 0.798
gmean-min: 0.794
2delikatne-cut ('tree', 'OVO', None, None)
g-mean: 0.693
gmean-min: 0.607
2delikatne-cut ('tree', 'OVO', None, 'acc')
g-mean: 0.723
gmean-min: 0.651
2delikatne-cut ('tree', 'OVO', None, 'avg_tpr_min')
g-mean: 0.712
gmean-min: 0.636
2delikatne-cut ('tree', 'OVO', 'globalCS', None)
g-mean: 0.703
gmean-min: 0.621
2delikatne-cut ('tree', 'OVO', 'globalCS', 'acc')
g-mean: 0.707
gmean-min: 0.628
2delikatne-cut ('tree', 'OVO', 'globalCS', 'avg_tpr_min')
g-mean: 0.699

g-mean: 0.694
gmean-min: 0.862
2delikatne-cut ('NB', 'dense', None, None)
g-mean: 0.52
gmean-min: 0.379
2delikatne-cut ('NB', 'dense', None, 'acc')
g-mean: 0.532
gmean-min: 0.391
2delikatne-cut ('NB', 'dense', None, 'avg_tpr_min')
g-mean: 0.537
gmean-min: 0.397
2delikatne-cut ('NB', 'dense', 'globalCS', None)
g-mean: 0.777
gmean-min: 0.733
2delikatne-cut ('NB', 'dense', 'globalCS', 'acc')
g-mean: 0.75
gmean-min: 0.702
2delikatne-cut ('NB', 'dense', 'globalCS', 'avg_tpr_min')
g-mean: 0.752
gmean-min: 0.702
2delikatne-cut ('NB', 'dense', 'SMOTE', None)
g-mean: 0.748
gmean-min: 0.689
2delikatne-cut ('NB', 'dense', 'SMOTE', 'acc')
g-mean: 0.749
gmean-min: 0.698
2delikatne-cut ('NB', 'dense', 'SMOTE', 'avg_tpr_min')
g-mean: 0.753
gmean-min: 0.698
2delikatne-cut ('NB', 'dense', 'SOUP', None)
g-mean: 0.759
gmean-min: 0.712
2delikatne-cut ('NB', 'dense', 'SOUP', 'acc')
g-mean: 0.745
gmean-min: 0.701
2delikatne-cut ('NB', 'dense', 'SOUP', 'avg_tpr_min')
g-mean: 0.747
gmean-min: 0.699
2delikatne

g-mean: 0.486
gmean-min: 0.372
3mocniej-cut ('tree', 'OVA', 'SMOTE', 'acc')
g-mean: 0.492
gmean-min: 0.41
3mocniej-cut ('tree', 'OVA', 'SMOTE', 'avg_tpr_min')
g-mean: 0.462
gmean-min: 0.374
3mocniej-cut ('tree', 'OVA', 'SOUP', None)
g-mean: 0.611
gmean-min: 0.553
3mocniej-cut ('tree', 'OVA', 'SOUP', 'acc')
g-mean: 0.612
gmean-min: 0.584
3mocniej-cut ('tree', 'OVA', 'SOUP', 'avg_tpr_min')
g-mean: 0.542
gmean-min: 0.477
3mocniej-cut ('KNN', 'sparse', None, None)
g-mean: 0.436
gmean-min: 0.31
3mocniej-cut ('KNN', 'sparse', None, 'acc')
g-mean: 0.435
gmean-min: 0.309
3mocniej-cut ('KNN', 'sparse', None, 'avg_tpr_min')
g-mean: 0.521
gmean-min: 0.466
3mocniej-cut ('KNN', 'sparse', 'globalCS', None)
g-mean: 0.526
gmean-min: 0.452
3mocniej-cut ('KNN', 'sparse', 'globalCS', 'acc')
g-mean: 0.52
gmean-min: 0.446
3mocniej-cut ('KNN', 'sparse', 'globalCS', 'avg_tpr_min')
g-mean: 0.476
gmean-min: 0.539
3mocniej-cut ('KNN', 'sparse', 'SMOTE', None)
g-mean: 0.499
gmean-min: 0.436
3mocniej-cut ('KNN', 

...

g-mean: 0.442
gmean-min: 0.499
new_winequality-red ('KNN', 'OVO', 'SOUP', 'avg_tpr_min')
g-mean: 0.394
gmean-min: 0.506
new_winequality-red ('KNN', 'OVA', None, None)
g-mean: 0.292
gmean-min: 0.157
new_winequality-red ('KNN', 'OVA', None, 'acc')
g-mean: 0.286
gmean-min: 0.143
new_winequality-red ('KNN', 'OVA', None, 'avg_tpr_min')
g-mean: 0.375
gmean-min: 0.248
new_winequality-red ('KNN', 'OVA', 'globalCS', None)
g-mean: 0.356
gmean-min: 0.23
new_winequality-red ('KNN', 'OVA', 'globalCS', 'acc')
g-mean: 0.435
gmean-min: 0.365
new_winequality-red ('KNN', 'OVA', 'globalCS', 'avg_tpr_min')
g-mean: 0.412
gmean-min: 0.317
new_winequality-red ('KNN', 'OVA', 'SMOTE', None)
g-mean: 0.275
gmean-min: 0.154
new_winequality-red ('KNN', 'OVA', 'SMOTE', 'acc')
g-mean: 0.438
gmean-min: 0.415
new_winequality-red ('KNN', 'OVA', 'SMOTE', 'avg_tpr_min')
g-mean: 0.404
gmean-min: 0.333
new_winequality-red ('KNN', 'OVA', 'SOUP', None)
g-mean: 0.309
gmean-min: 0.175
new_winequality-red ('KNN', 'OVA', 'SOUP',

g-mean: 0.313
gmean-min: 0.201
new_yeast ('tree', 'complete', None, 'avg_tpr_min')
g-mean: 0.29
gmean-min: 0.184
new_yeast ('tree', 'complete', 'globalCS', None)
g-mean: 0.246
gmean-min: 0.123
new_yeast ('tree', 'complete', 'globalCS', 'acc')
g-mean: 0.267
gmean-min: 0.142
new_yeast ('tree', 'complete', 'globalCS', 'avg_tpr_min')
g-mean: 0.287
gmean-min: 0.178
new_yeast ('tree', 'complete', 'SMOTE', None)
g-mean: 0.245
gmean-min: 0.122
new_yeast ('tree', 'complete', 'SMOTE', 'acc')
g-mean: 0.274
gmean-min: 0.145
new_yeast ('tree', 'complete', 'SMOTE', 'avg_tpr_min')
g-mean: 0.274
gmean-min: 0.145
new_yeast ('tree', 'complete', 'SOUP', None)
g-mean: 0.343
gmean-min: 0.225
new_yeast ('tree', 'complete', 'SOUP', 'acc')
g-mean: 0.292
gmean-min: 0.156
new_yeast ('tree', 'complete', 'SOUP', 'avg_tpr_min')
g-mean: 0.343
gmean-min: 0.228
new_yeast ('tree', 'OVO', None, None)
g-mean: 0.248
gmean-min: 0.133
new_yeast ('tree', 'OVO', None, 'acc')
g-mean: 0.236
gmean-min: 0.118
new_yeast ('tree', 

g-mean: 0.018
gmean-min: 0.002
new_yeast ('NB', 'dense', None, 'avg_tpr_min')
g-mean: 0.081
gmean-min: 0.132
new_yeast ('NB', 'dense', 'globalCS', None)
g-mean: 0.083
gmean-min: 0.143
new_yeast ('NB', 'dense', 'globalCS', 'acc')
g-mean: 0.023
gmean-min: 0.004
new_yeast ('NB', 'dense', 'globalCS', 'avg_tpr_min')
g-mean: 0.058
gmean-min: 0.105
new_yeast ('NB', 'dense', 'SMOTE', None)
g-mean: 0.114
gmean-min: 0.173
new_yeast ('NB', 'dense', 'SMOTE', 'acc')
g-mean: 0.023
gmean-min: 0.003
new_yeast ('NB', 'dense', 'SMOTE', 'avg_tpr_min')
g-mean: 0.098
gmean-min: 0.157
new_yeast ('NB', 'dense', 'SOUP', None)
g-mean: 0.085
gmean-min: 0.088
new_yeast ('NB', 'dense', 'SOUP', 'acc')
g-mean: 0.019
gmean-min: 0.004
new_yeast ('NB', 'dense', 'SOUP', 'avg_tpr_min')
g-mean: 0.058
gmean-min: 0.12
new_yeast ('NB', 'complete', None, None)
g-mean: 0.102
gmean-min: 0.043
new_yeast ('NB', 'complete', None, 'acc')
g-mean: 0.074
gmean-min: 0.022
new_yeast ('NB', 'complete', None, 'avg_tpr_min')
g-mean: 0.164

g-mean: 0.926
gmean-min: 0.905
thyroid-newthyroid ('tree', 'OVA', 'SMOTE', 'avg_tpr_min')
g-mean: 0.905
gmean-min: 0.871
thyroid-newthyroid ('tree', 'OVA', 'SOUP', None)
g-mean: 0.923
gmean-min: 0.894
thyroid-newthyroid ('tree', 'OVA', 'SOUP', 'acc')
g-mean: 0.93
gmean-min: 0.934
thyroid-newthyroid ('tree', 'OVA', 'SOUP', 'avg_tpr_min')
g-mean: 0.927
gmean-min: 0.922
thyroid-newthyroid ('KNN', 'sparse', None, None)
g-mean: 0.903
gmean-min: 0.869
thyroid-newthyroid ('KNN', 'sparse', None, 'acc')
g-mean: 0.862
gmean-min: 0.812
thyroid-newthyroid ('KNN', 'sparse', None, 'avg_tpr_min')
g-mean: 0.739
gmean-min: 0.887
thyroid-newthyroid ('KNN', 'sparse', 'globalCS', None)
g-mean: 0.93
gmean-min: 0.914
thyroid-newthyroid ('KNN', 'sparse', 'globalCS', 'acc')
g-mean: 0.92
gmean-min: 0.901
thyroid-newthyroid ('KNN', 'sparse', 'globalCS', 'avg_tpr_min')
g-mean: 0.771
gmean-min: 0.927
thyroid-newthyroid ('KNN', 'sparse', 'SMOTE', None)
g-mean: 0.952
gmean-min: 0.946
thyroid-newthyroid ('KNN', 'spa

g-mean: 0.887
gmean-min: 0.843
thyroid-newthyroid ('NB', 'complete', 'SOUP', 'acc')
g-mean: 0.897
gmean-min: 0.87
thyroid-newthyroid ('NB', 'complete', 'SOUP', 'avg_tpr_min')
g-mean: 0.892
gmean-min: 0.858
thyroid-newthyroid ('NB', 'OVO', None, None)
g-mean: 0.943
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', None, 'acc')
g-mean: 0.943
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', None, 'avg_tpr_min')
g-mean: 0.943
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'globalCS', None)
g-mean: 0.941
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'globalCS', 'acc')
g-mean: 0.941
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'globalCS', 'avg_tpr_min')
g-mean: 0.941
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'SMOTE', None)
g-mean: 0.941
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'SMOTE', 'acc')
g-mean: 0.939
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'SMOTE', 'avg_tpr_min')
g-mean: 0.939
gmean-min: 0.927
thyroid-newthyroid ('NB', 'OVO', 'SOUP', None)
g-mean

In [7]:
df_gmean = pd.DataFrame(results_g_mean)
df_gmean_min = pd.DataFrame(results_gmean_min)

for df in (df_gmean, df_gmean_min):
    df.set_index('dataset')
    df.columns = pd.MultiIndex.from_tuples(
        [('classifier', 'encoding', 'preprocessing', 'weights')] + [(str(m[0]), str(m[1]), str(m[2]), str(m[3])) for m in methods])
    
import os
import datetime
    
current_date = str(datetime.datetime.today())[:-7].replace(' ', '_').replace(':','_')

directory='ECOC-' + current_date
os.makedirs(directory)

df_gmean.to_csv(f'./{directory}/ecoc_kfold_gmean.csv', index=False)
df_gmean_min.to_csv(f'./{directory}/ecoc_kfold_gmean_min.csv', index=False)


## G-mean

In [31]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 4000
df_gmean

classifier   tree                                     \
                       encoding sparse                                      
                   oversampling   None                    globalCS          
                        weights   None    acc avg_tpr_min     None    acc   
0              arff\1czysty-cut  0.943  0.939       0.760    0.946  0.942   
1           arff\2delikatne-cut  0.712  0.713       0.659    0.719  0.716   
2             arff\3mocniej-cut  0.501  0.493       0.533    0.503  0.484   
3   arff\4delikatne-bezover-cut  0.772  0.778       0.700    0.772  0.780   
4            arff\balance-scale  0.122  0.198       0.098    0.116  0.167   
5                      arff\car  0.172  0.542       0.549    0.174  0.544   
6                arff\cleveland  0.023  0.070       0.081    0.023  0.075   
7             arff\cleveland_v2  0.039  0.062       0.134    0.039  0.043   
8                      arff\cmc  0.452  0.443       0.200    0.449  0.449   
9              arff\dermatology  0.949  0.962       0.908    0.954  0.960   
10                   arff\flare  0.452  0.443       0.480    0.455  0.473   
11                   arff\glass  0.601  0.527       0.575    0.611  0.490   
12              arff\hayes-roth  0.841  0.840       0.100    0.843  0.841   
13               arff\new_ecoli  0.312  0.724       0.741    0.290  0.730   
14           arff\new_led7digit  0.739  0.754       0.756    0.731  0.750   
15             arff\new_vehicle  0.897  0.890       0.892    0.892  0.886   
16     arff\new_winequality-red  0.095  0.315       0.441    0.088  0.290   
17               arff\new_yeast  0.123  0.229       0.246    0.122  0.220   
18      arff\thyroid-newthyroid  0.903  0.894       0.710    0.901  0.900   

                                                                           \
                                                                    dense   
                SMOTE                      SOUP                      None   
   avg_tpr_min   None    acc avg_tpr_min   None    acc avg_tpr_min   None   
0        0.764  0.955  0.952       0.765  0.955  0.952       0.754  0.931   
1        0.668  0.750  0.741       0.670  0.800  0.799       0.706  0.702   
2        0.534  0.511  0.513       0.515  0.604  0.609       0.520  0.472   
3        0.704  0.809  0.805       0.718  0.889  0.884       0.717  0.759   
4        0.082  0.169  0.284       0.140  0.473  0.489       0.207  0.108   
5        0.566  0.174  0.556       0.577  0.154  0.622       0.613  0.916   
6        0.075  0.027  0.067       0.082  0.025  0.068       0.102  0.079   
7        0.119  0.058  0.050       0.140  0.051  0.113       0.171  0.078   
8        0.182  0.448  0.447       0.150  0.485  0.474       0.109  0.421   
9        0.906  0.949  0.961       0.887  0.965  0.962       0.922  0.953   
10       0.486  0.435  0.438       0.455  0.290  0.262       0.436  0.440   
11       0.525  0.627  0.560       0.591  0.558  0.624       0.592  0.512   
12       0.100  0.840  0.827       0.106  0.810  0.750       0.358  0.755   
13       0.751  0.283  0.744       0.747  0.206  0.746       0.738  0.738   
14       0.752  0.739  0.756       0.758  0.743  0.763       0.755  0.762   
15       0.883  0.899  0.890       0.890  0.865  0.875       0.876  0.889   
16       0.460  0.097  0.342       0.468  0.102  0.407       0.478  0.368   
17       0.260  0.125  0.252       0.304  0.131  0.277       0.333  0.267   
18       0.703  0.904  0.910       0.710  0.927  0.904       0.725  0.893   

                                                                             \
                                                                              
                      globalCS                     SMOTE                      
      acc avg_tpr_min     None    acc avg_tpr_min   None    acc avg_tpr_min   
0   0.935       0.935    0.937  0.940       0.940  0.943  0.944       0.945   
1   0.697       0.693    0.702  0.701       0.697  0.709  0.723       0.720   
2  

## G-mean for minority

In [32]:
df_gmean_min

classifier   tree                                     \
                       encoding sparse                                      
                   oversampling   None                    globalCS          
                        weights   None    acc avg_tpr_min     None    acc   
0              arff\1czysty-cut  0.930  0.926       0.968    0.934  0.930   
1           arff\2delikatne-cut  0.639  0.634       0.758    0.644  0.637   
2             arff\3mocniej-cut  0.402  0.384       0.525    0.402  0.373   
3   arff\4delikatne-bezover-cut  0.725  0.731       0.880    0.723  0.731   
4            arff\balance-scale  0.011  0.040       0.464    0.009  0.031   
5                      arff\car  0.948  0.911       0.931    0.962  0.929   
6                arff\cleveland  0.051  0.058       0.068    0.052  0.065   
7             arff\cleveland_v2  0.016  0.032       0.088    0.016  0.020   
8                      arff\cmc  0.361  0.357       0.473    0.373  0.380   
9              arff\dermatology  0.985  0.985       0.995    0.980  0.985   
10                   arff\flare  0.314  0.302       0.385    0.329  0.354   
11                   arff\glass  0.683  0.458       0.553    0.646  0.398   
12              arff\hayes-roth  0.829  0.815       1.000    0.825  0.817   
13               arff\new_ecoli  0.667  0.657       0.721    0.667  0.663   
14           arff\new_led7digit  0.658  0.697       0.702    0.660  0.703   
15             arff\new_vehicle  0.889  0.881       0.887    0.885  0.876   
16     arff\new_winequality-red  0.013  0.158       0.345    0.012  0.142   
17               arff\new_yeast  0.112  0.125       0.132    0.111  0.120   
18      arff\thyroid-newthyroid  0.876  0.867       0.915    0.877  0.871   

                                                                           \
                                                                    dense   
                SMOTE                      SOUP                      None   
   avg_tpr_min   None    acc avg_tpr_min   None    acc avg_tpr_min   None   
0        0.974  0.951  0.947       0.980  0.972  0.972       0.991  0.913   
1        0.762  0.698  0.676       0.772  0.797  0.795       0.817  0.622   
2        0.522  0.464  0.437       0.555  0.601  0.602       0.638  0.362   
3        0.890  0.789  0.776       0.932  0.954  0.951       0.990  0.708   
4        0.547  0.027  0.076       0.446  0.225  0.274       0.541  0.007   
5        0.938  0.972  0.928       0.938  0.827  0.967       0.970  0.868   
6        0.065  0.060  0.058       0.073  0.059  0.075       0.093  0.049   
7        0.075  0.026  0.026       0.095  0.024  0.071       0.121  0.039   
8        0.476  0.352  0.375       0.485  0.508  0.494       0.589  0.379   
9        0.995  0.985  0.985       0.995  0.985  0.965       0.965  0.965   
10       0.419  0.280  0.284       0.367  0.320  0.249       0.694  0.286   
11       0.506  0.703  0.496       0.608  0.736  0.685       0.716  0.402   
12       1.000  0.828  0.798       1.000  0.755  0.700       0.865  0.805   
13       0.732  0.667  0.696       0.745  0.772  0.795       0.842  0.671   
14       0.708  0.660  0.700       0.710  0.691  0.733       0.739  0.705   
15       0.886  0.898  0.887       0.893  0.902  0.933       0.935  0.879   
16       0.372  0.014  0.192       0.396  0.017  0.288       0.460  0.220   
17       0.151  0.118  0.149       0.198  0.113  0.169       0.224  0.153   
18       0.917  0.889  0.890       0.942  0.928  0.898       0.939  0.866   

                                                                             \
                                                                              
                      globalCS                     SMOTE                      
      acc avg_tpr_min     None    acc avg_tpr_min   None    acc avg_tpr_min   
0   0.923       0.923    0.922  0.931       0.931  0.932  0.938       0.937   
1   0.617       0.611    0.619  0.622       0.615  0.632  0.652       0.646   
2  

In [33]:
ranks_gmean = dict()
ranks_gmean_min = dict()

for results, ranks in zip((results_g_mean, results_gmean_min), (ranks_gmean, ranks_gmean_min)):
    for ds_idx, ds in enumerate(datasets_names):
        ranks[ds] = dict()
        for bc in binary_classifiers:
            ranks[ds][bc] = rankdata(
                [-results[method][ds_idx] for method in methods if method[0]==bc])

            
avg_ranks_gmean=dict()
avg_ranks_gmean_min =dict()

for avg_ranks, ranks in zip((avg_ranks_gmean, avg_ranks_gmean_min), (ranks_gmean, ranks_gmean_min)):
    for bc in binary_classifiers:
        avg_ranks[bc]=dict()
        for method_idx, method in enumerate([m for m in methods if m[0]==bc]):
            avg_ranks[bc][method] = round(np.mean([ranks[ds][bc][method_idx] for ds in datasets_names]),3)
            
for avg_ranks in (avg_ranks_gmean, avg_ranks_gmean_min):
    for bc in binary_classifiers:
        for method in methods:
            if method[0]==bc:
                avg_ranks[bc][str(method[1:])] = avg_ranks[bc][method]
                avg_ranks[bc].pop(method)

df_gmean_avg_ranks = pd.DataFrame(avg_ranks_gmean).T
df_gmean_min_avg_ranks = pd.DataFrame(avg_ranks_gmean_min).T

## Mean ranks in G-mean:

In [34]:
df_gmean_avg_ranks.T.sort_values('tree')

,tree,KNN,NB
"('OVO', 'SOUP', None)",10.816,9.605,11.842
"('OVO', 'SOUP', 'acc')",13.184,19.000,14.711
"('complete', 'SOUP', 'acc')",14.158,25.579,31.763
"('complete', 'SOUP', 'avg_tpr_min')",15.632,25.237,28.632
"('complete', 'SOUP', None)",19.526,27.868,34.842
"('complete', 'SMOTE', 'acc')",19.947,20.658,27.132
"('dense', 'SOUP', 'avg_tpr_min')",20.000,23.211,34.132
"('complete', 'SMOTE', 'avg_tpr_min')",20.684,23.605,22.316
"('sparse', 'SOUP', 'acc')",21.026,28.105,25.500
"('dense', 'SOUP', 'acc')",21.184,25.132,33.211


## Mean ranks in G-mean for minority

In [35]:
df_gmean_min_avg_ranks.T.sort_values('tree')

,tree,KNN,NB
"('sparse', 'SOUP', 'avg_tpr_min')",3.947,7.895,10.526
"('OVO', 'SOUP', 'avg_tpr_min')",10.211,7.974,11.526
"('sparse', 'SMOTE', 'avg_tpr_min')",10.789,14.658,12.605
"('OVO', 'SOUP', None)",11.316,7.447,13.289
"('OVO', 'SOUP', 'acc')",11.342,11.632,17.947
"('sparse', 'globalCS', 'avg_tpr_min')",14.289,11.658,11.158
"('sparse', 'SOUP', 'acc')",15.868,21.632,25.526
"('complete', 'SOUP', 'avg_tpr_min')",16.184,23.605,28.605
"('sparse', None, 'avg_tpr_min')",16.447,28.421,21.921
"('dense', 'SOUP', 'avg_tpr_min')",18.105,25.211,32.500


## Mean results

In [36]:
mean_results = dict()
for m in methods:
    mean_results[str(m)] = [round(np.mean(results_g_mean[m]),2), round(np.mean(results_gmean_min[m]),2)]

    
df_mean = pd.DataFrame(mean_results).T
df_mean.columns = 'gmean gmean_min'.split()

df_mean.sort_values('gmean', ascending=False)

,gmean,gmean_min
"('KNN', 'OVO', 'SOUP', None)",0.66,0.68
"('tree', 'OVO', 'SOUP', None)",0.66,0.64
"('tree', 'OVO', 'SOUP', 'acc')",0.65,0.63
"('tree', 'complete', 'SOUP', 'avg_tpr_min')",0.64,0.60
"('KNN', 'OVO', 'SOUP', 'acc')",0.63,0.65
"('tree', 'complete', 'SOUP', 'acc')",0.63,0.59
"('tree', 'dense', 'SOUP', 'avg_tpr_min')",0.63,0.59
"('KNN', 'OVO', 'globalCS', None)",0.62,0.59
"('tree', 'dense', 'SOUP', 'acc')",0.62,0.58
"('tree', 'dense', 'SMOTE', 'avg_tpr_min')",0.61,0.54


In [37]:
def get_weights_results(results, compared):
    methods_without_weight=[(bc,encoding, bo) for bc in binary_classifiers for encoding in encodings for bo in binary_oversamplings]
    weights_results = dict()
    for m in methods_without_weight:
        compared_method = str(m)
        weights_results[compared_method] = dict()
        wins, ties, defeats = 0, 0, 0
        without_weights = m + (None,)
        with_weights = m + (compared,)
        for ds_idx in range(len(datasets_names)):
            if (results[with_weights][ds_idx]==0 and results[without_weights][ds_idx]==0) or (min(results[with_weights][ds_idx],results[without_weights][ds_idx]) !=0 and np.abs(results[with_weights][ds_idx] - results[without_weights][ds_idx]) / min(results[with_weights][ds_idx],results[without_weights][ds_idx]) <= 0.01):
                ties += 1
            elif results[with_weights][ds_idx] > results[without_weights][ds_idx]:
                wins += 1
            else: 
                defeats += 1
        weights_results[compared_method]['wins'], weights_results[compared_method]['ties'], weights_results[compared_method]['defeats'] = wins, ties, defeats

    df = pd.DataFrame(weights_results).T
    return df

## wins - ties(no more than 1% difference) -defeats in G-mean with vs without weights

### acc weights vs no weights

In [38]:
df_gm = get_weights_results(results_g_mean, 'acc')
df_gm

,wins,ties,defeats
"('tree', 'sparse', None)",9,5,5
"('tree', 'sparse', 'globalCS')",10,7,2
"('tree', 'sparse', 'SMOTE')",8,6,5
"('tree', 'sparse', 'SOUP')",10,5,4
"('tree', 'dense', None)",7,10,2
"('tree', 'dense', 'globalCS')",7,4,8
"('tree', 'dense', 'SMOTE')",7,7,5
"('tree', 'dense', 'SOUP')",7,6,6
"('tree', 'complete', None)",12,6,1
"('tree', 'complete', 'globalCS')",10,6,3


###  avgtprmin weights vs no weights

In [39]:
df_gm1 = get_weights_results(results_g_mean, 'avg_tpr_min')
df_gm1

,wins,ties,defeats
"('tree', 'sparse', None)",9,1,9
"('tree', 'sparse', 'globalCS')",9,0,10
"('tree', 'sparse', 'SMOTE')",8,1,10
"('tree', 'sparse', 'SOUP')",10,0,9
"('tree', 'dense', None)",8,7,4
"('tree', 'dense', 'globalCS')",7,4,8
"('tree', 'dense', 'SMOTE')",8,7,4
"('tree', 'dense', 'SOUP')",7,6,6
"('tree', 'complete', None)",13,5,1
"('tree', 'complete', 'globalCS')",10,6,3


## G-mean for minority

### acc weights vs no weights

In [40]:
df_gmm = get_weights_results(results_gmean_min, 'acc')
df_gmm

,wins,ties,defeats
"('tree', 'sparse', None)",6,5,8
"('tree', 'sparse', 'globalCS')",9,5,5
"('tree', 'sparse', 'SMOTE')",7,4,8
"('tree', 'sparse', 'SOUP')",9,4,6
"('tree', 'dense', None)",10,6,3
"('tree', 'dense', 'globalCS')",6,4,9
"('tree', 'dense', 'SMOTE')",7,4,8
"('tree', 'dense', 'SOUP')",11,3,5
"('tree', 'complete', None)",14,3,2
"('tree', 'complete', 'globalCS')",11,4,4


### avgtprmin weights vs no weights

In [41]:
df_gmm1 = get_weights_results(results_gmean_min, 'avg_tpr_min')
df_gmm1

,wins,ties,defeats
"('tree', 'sparse', None)",16,1,2
"('tree', 'sparse', 'globalCS')",16,1,2
"('tree', 'sparse', 'SMOTE')",16,1,2
"('tree', 'sparse', 'SOUP')",17,0,2
"('tree', 'dense', None)",12,2,5
"('tree', 'dense', 'globalCS')",9,4,6
"('tree', 'dense', 'SMOTE')",9,7,3
"('tree', 'dense', 'SOUP')",11,3,5
"('tree', 'complete', None)",15,2,2
"('tree', 'complete', 'globalCS')",11,6,2
